In [1]:
import pandas as pd

In [2]:
a = pd.read_csv("../기상청공모전/data/derivate_date_fillna.csv")

In [3]:
a

,Unnamed: 0,anchor_drag,num,latitude,longitude,sog,cog,hdg,d_lati,d_long,d_d_lati,d_d_long
0,2,0,1001,35.461472,129.439878,0.4,9.6,273.0,0.000312,0.000333,0.000342,0.000342
1,3,0,1001,35.461688,129.439792,0.2,328.0,267.0,0.000216,-0.000086,-0.000096,-0.000096
2,4,0,1001,35.461708,129.439733,0.2,260.1,265.0,0.000020,-0.000059,-0.000196,-0.000196
3,5,0,1001,35.461720,129.439708,0.1,262.8,270.0,0.000012,-0.000025,-0.000008,-0.000008
4,6,0,1001,35.461733,129.439717,0.3,277.6,273.0,0.000013,0.000009,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...
681435,682471,1,4087,35.053083,129.061333,1.3,26.0,97.0,0.000033,0.000016,-0.000017,-0.000017
681436,682472,1,4087,35.053083,129.061333,1.3,29.0,99.0,0.000000,0.000000,-0.000033,-0.000033
681437,682473,1,4087,35.053117,129.061400,1.2,48.0,106.0,0.000034,0.000067,0.000034,0.000034
681438,682474,1,4087,35.053117,129.061417,1.2,52.0,108.0,0.000000,0.000017,-0.000034,-0.000034


In [4]:
a1 = a.iloc[:,1:]

In [5]:
a1

,anchor_drag,num,latitude,longitude,sog,cog,hdg,d_lati,d_long,d_d_lati,d_d_long
0,0,1001,35.461472,129.439878,0.4,9.6,273.0,0.000312,0.000333,0.000342,0.000342
1,0,1001,35.461688,129.439792,0.2,328.0,267.0,0.000216,-0.000086,-0.000096,-0.000096
2,0,1001,35.461708,129.439733,0.2,260.1,265.0,0.000020,-0.000059,-0.000196,-0.000196
3,0,1001,35.461720,129.439708,0.1,262.8,270.0,0.000012,-0.000025,-0.000008,-0.000008
4,0,1001,35.461733,129.439717,0.3,277.6,273.0,0.000013,0.000009,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...,...
681435,1,4087,35.053083,129.061333,1.3,26.0,97.0,0.000033,0.000016,-0.000017,-0.000017
681436,1,4087,35.053083,129.061333,1.3,29.0,99.0,0.000000,0.000000,-0.000033,-0.000033
681437,1,4087,35.053117,129.061400,1.2,48.0,106.0,0.000034,0.000067,0.000034,0.000034
681438,1,4087,35.053117,129.061417,1.2,52.0,108.0,0.000000,0.000017,-0.000034,-0.000034


# XGB

In [6]:
# XGBoost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열


# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 데이터셋 생성
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# XGBoost 모델 학습
params = {'objective': 'binary:logistic', 'eval_metric': 'logloss'}
num_rounds = 100  # 반복 횟수 설정

model = xgb.train(params, dtrain, num_rounds)

# 테스트 데이터 예측
y_pred = model.predict(dtest)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
print("정확도:", accuracy)

정확도: 0.9944749354308523


In [7]:
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred_binary))

0.8591733682438751


In [8]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 계산
tp = cm[1, 1]  # True Positive
tn = cm[0, 0]  # True Negative
fp = cm[0, 1]  # False Positive
fn = cm[1, 0]  # False Negative

csi = tp / (tp + fn + fp)
print("CSI 점수:", csi)

CSI 점수: 0.7531147540983607


# LightGBM

In [10]:
# LightGBM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)

# LightGBM 모델 학습
params = {'objective': 'binary', 'metric': 'binary_logloss'}
num_rounds = 100  # 반복 횟수 설정

model = lgb.train(params, train_data, num_rounds)

# 테스트 데이터 예측
y_pred = model.predict(X_test)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
print("정확도:", accuracy)

[LightGBM] [Info] Number of positive: 11641, number of negative: 533511
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2154
[LightGBM] [Info] Number of data points in the train set: 545152, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.021354 -> initscore=-3.824946
[LightGBM] [Info] Start training from score -3.824946
정확도: 0.9908869452923221


In [11]:
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred_binary))

0.7617951668584579


In [12]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 계산
tp = cm[1, 1]  # True Positive
tn = cm[0, 0]  # True Negative
fp = cm[0, 1]  # False Positive
fn = cm[1, 0]  # False Negative

csi = tp / (tp + fn + fp)
print("CSI 점수:", csi)

CSI 점수: 0.6152416356877324


# RandomForest

In [13]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForest 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.9941741019018548


In [14]:
from sklearn.metrics import f1_score

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# F1 score 계산
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

F1 score: 0.8458850931677019


In [15]:
def calculate_csi(y_true, y_pred):
    # True Positive (TP): 예측과 실제 값이 모두 1인 경우
    tp = sum((y_true == 1) & (y_pred == 1))
    
    # False Positive (FP): 예측은 1이지만 실제 값은 0인 경우
    fp = sum((y_true == 0) & (y_pred == 1))
    
    # False Negative (FN): 예측은 0이지만 실제 값은 1인 경우
    fn = sum((y_true == 1) & (y_pred == 0))
    
    # Critical Success Index (CSI): TP / (TP + FP + FN)
    csi = tp / (tp + fp + fn)
    
    return csi

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# CSI 점수 계산
csi = calculate_csi(y_test, y_pred)
print("CSI 점수:", csi)

CSI 점수: 0.7329297006390851


# CatBoost

In [16]:
# CatBoost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost 모델 학습
model = CatBoostClassifier(iterations=100, eval_metric='Logloss')
model.fit(X_train, y_train, verbose=False)

# 테스트 데이터 예측
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # 클래스 1에 대한 예측 확률 추출

# 이진 분류에서의 예측 결과 변환
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred_proba]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
print("정확도:", accuracy)

# F1 점수 계산
f1 = f1_score(y_test, y_pred_binary)
print("F1 점수:", f1)

정확도: 0.9914886123503169
F1 점수: 0.7710225029609159


In [17]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 점수 계산
tn, fp, fn, tp = cm.ravel()
csi = tp / (tp + fp + fn)
print("CSI 점수:", csi)

CSI 점수: 0.6273690973337617
